# Scratch

In [2]:
EMPFILE = "/Users/adarshrp/Projects/flare/data/emp.csv"
emp = spark.read.option("header", True).option("inferSchema", True).csv(EMPFILE)
emp.printSchema()
emp.createOrReplaceTempView("emp")


root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dept_id: integer (nullable = true)



In [15]:
spark.sql("""
select name from EMP
where age > ANY (select max(age) from EMP)
""").show(10)

ParseException: 
mismatched input '(' expecting {<EOF>, ';'}(line 3, pos 16)

== SQL ==

select name from EMP
where age > ANY (select max(age) from EMP)
----------------^^^


In [30]:
spark.sql("""
WITH CTE1 AS (select '--cte1', age, '--cte1' from emp),
 CTE2 AS (select '--cte2', age, '--cte2' from emp)
SELECT * FROM CTE1, (select '2nd', dept_id, name from EMP) CTE1
WHERE name in (select name from emp CTE1 where CTE1.age >= 0) 
and age in (select CTE2.age from EMP CTE2 where CTE1.age = CTE2.age)
""").show(100)

+------+---+------+---+-------+-----+
|--cte1|age|--cte1|2nd|dept_id| name|
+------+---+------+---+-------+-----+
|--cte1| 32|--cte1|2nd|      1| john|
|--cte1| 30|--cte1|2nd|      1| john|
|--cte1| 53|--cte1|2nd|      1| john|
|--cte1| 45|--cte1|2nd|      1| john|
|--cte1| 25|--cte1|2nd|      1| john|
|--cte1| 28|--cte1|2nd|      1| john|
|--cte1| 45|--cte1|2nd|      1| john|
|--cte1| 25|--cte1|2nd|      1| john|
|--cte1| 28|--cte1|2nd|      1| john|
|--cte1| 28|--cte1|2nd|      1| john|
|--cte1| 24|--cte1|2nd|      1| john|
|--cte1| 27|--cte1|2nd|      1| john|
|--cte1| 42|--cte1|2nd|      1| john|
|--cte1| 38|--cte1|2nd|      1| john|
|--cte1| 61|--cte1|2nd|      1| john|
|--cte1| 32|--cte1|2nd|      2|  aby|
|--cte1| 30|--cte1|2nd|      2|  aby|
|--cte1| 53|--cte1|2nd|      2|  aby|
|--cte1| 45|--cte1|2nd|      2|  aby|
|--cte1| 25|--cte1|2nd|      2|  aby|
|--cte1| 28|--cte1|2nd|      2|  aby|
|--cte1| 45|--cte1|2nd|      2|  aby|
|--cte1| 25|--cte1|2nd|      2|  aby|
|--cte1| 28|

In [35]:
parquetFile = spark.read.parquet("/tmp/file.parquet")
parquetFile.createOrReplaceTempView("parquetFile")

In [36]:
parquetFile

DataFrame[column_1: bigint, column_2: string, column_3: string]

In [37]:
parquetFile.columns

['column_1', 'column_2', 'column_3']

In [38]:
spark.sql("SELECT * FROM parquetFile").show(100)

+--------+-----------+--------------------+
|column_1|   column_2|            column_3|
+--------+-----------+--------------------+
|       1|    AMERICA|hs use ironic, ev...|
|       2|       ASIA|ges. thinly even ...|
|       3|     EUROPE|ly final courts c...|
|       4|MIDDLE EAST|uickly special ac...|
+--------+-----------+--------------------+



In [41]:
parquetFile.rdd.getNumPartitions()

1

In [42]:
parquetFile.write.format("parquet").partitionBy("column_2").mode("overwrite").save("/tmp/pqptns.parquet")